In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [30]:
classes = np.array([str(l) for l in range(28)])
df = pd.read_csv('aug_train.csv'); len(df)
PATH = Path('./')
SAMPLE = Path('sample_submission.csv')
sample_sub = pd.read_csv(PATH/SAMPLE)
test_names = list(sample_sub.Id.values)

In [29]:
def count_classes(df):
    out = np.zeros(28, dtype=int)
    for i in range(len(df)):
        idxs = [int(x) for x in df.Target.iloc[i].split()]
        out[idxs]+=1
    return out

In [31]:
cc = count_classes(df)

In [36]:
hard_classes = np.arange(28)[cc<500]; hard_classes

array([ 8,  9, 10, 15, 17, 20, 27])

# Model ensmeble

In [3]:
model_name = 'ensemble'

In [4]:
r18c = np.load('r18c.npy')

In [5]:
r34 = np.load('predsR34.npy')

In [6]:
r18 = np.load('predsR18.npy')

In [19]:
ense = (0.33*r18 +0.33*r34 + 0.33*r18c)
ense2 = (r18+r34)/2

In [20]:
def preds2frame(preds, th=0.4):
    print(preds.shape)
    res = np.array([" ".join(classes[(np.where(pp>th))])for pp in preds])
    return pd.DataFrame(np.array([test_names, res]).T, columns = ['Id','Predicted'])

In [21]:
df_r18 = preds2frame(r18)
df_r34 = preds2frame(r34)
df_r18c = preds2frame(r18c)
df_ensemble = preds2frame(ense)
df_ensemble2 = preds2frame(ense2)

(11702, 28)
(11702, 28)
(11702, 28)
(11702, 28)
(11702, 28)


In [22]:
df = pd.concat([df_r18, 
                df_r34['Predicted'], 
                df_r18c['Predicted'], 
                df_ensemble['Predicted'],
                df_ensemble2['Predicted']],ignore_index=True, axis=1)
df.columns = ['Id', 'r18', 'r34', 'r18c', 'ense', 'ense2']

In [23]:
df.head(15)

,Id,r18,r34,r18c,ense,ense2
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,2,2,2,2,2
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,5 25,5,5 25,5 25,5 25
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0 5 25,0 5 21 25,0 5 18 21 25,0 5 21 25,0 5 25
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0 25,0 16 18 25,0 16 17 18,0 16 17 18 25,0 16 25
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,7,0,7,7,7
5,00109f6a-bac8-11e8-b2b7-ac1f6b6435d0,4,4,4,4,4
6,001765de-bacd-11e8-b2b8-ac1f6b6435d0,0 4 23 25,4 5 25,0 4 25,0 4 25,0 4 25
7,0018641a-bac9-11e8-b2b8-ac1f6b6435d0,0 25,0 6,0 6 23,0,0
8,00200f22-bad7-11e8-b2b9-ac1f6b6435d0,0,0 26,0 26,0 26,0
9,0026f154-bac6-11e8-b2b7-ac1f6b6435d0,0 2 25,2 4 21 25,2 21 25,2 21 25,2 21 25


In [24]:
idxs = np.where(df['ense'] != df['ense2'])

In [25]:
df.iloc[idxs]

,Id,r18,r34,r18c,ense,ense2
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0 5 25,0 5 21 25,0 5 18 21 25,0 5 21 25,0 5 25
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0 25,0 16 18 25,0 16 17 18,0 16 17 18 25,0 16 25
8,00200f22-bad7-11e8-b2b9-ac1f6b6435d0,0,0 26,0 26,0 26,0
10,002729d2-bace-11e8-b2b8-ac1f6b6435d0,25,18,18 25,18 25,25
22,007eca68-bac7-11e8-b2b7-ac1f6b6435d0,0 25,0 16 25,0 25,0 25,0 16 25
38,00bb8fa8-bada-11e8-b2b9-ac1f6b6435d0,0 21 25,25,0 21 25,0 21 25,21 25
39,00c19c92-bac9-11e8-b2b8-ac1f6b6435d0,0 1,1,1,1,0 1
52,0106ac3e-bac8-11e8-b2b7-ac1f6b6435d0,,20 26,20,20,
56,013868ca-bad0-11e8-b2b8-ac1f6b6435d0,0 25,17 25,0 25,0 25,0 17 25
59,014c1cac-bacc-11e8-b2b8-ac1f6b6435d0,25,21 25,25,25,21 25


In [26]:
df_ensemble.head()

,Id,Predicted
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,2
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,5 25
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0 5 21 25
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0 16 17 18 25
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,7


In [27]:
df_ensemble.to_csv(f'{model_name}.csv', index=False)

In [28]:
!kaggle competitions submit -c human-protein-atlas-image-classification -f "ensemble.csv" -m "(0.4*r18 +0.6*r34):  model ensemble"

100%|█████████████████████████████████████████| 474k/474k [00:01<00:00, 353kB/s]
Successfully submitted to Human Protein Atlas Image Classification

In [15]:
!kaggle competitions submit -c human-protein-atlas-image-classification -f "r18c.csv" -m "custom R18"

100%|█████████████████████████████████████████| 480k/480k [00:01<00:00, 340kB/s]
Successfully submitted to Human Protein Atlas Image Classification